In [1]:
import tensorflow as tf
import numpy as np

import model.cnn as cnn
import model.rnn as rnn
import model.classifier as classifier

import data.ops

import time
import matplotlib.pyplot as plt
%matplotlib inline

# Model definition

In [2]:
tf.reset_default_graph()
batch_size = tf.placeholder_with_default(32, [])
input_op, seq_len, label = data.ops.get_batch_producer(
    batch_size=batch_size, path='./data/train.TFRecord')

cnn_params = {
    'out_dims' : [10],
    'kernel_sizes' : 32,
    'pool_sizes' : 10
}
rnn_params = {
    'rnn_sizes' : [10],
    'time_steps' : 100
}
fc_params = {
    'fc_sizes' : []
}

c = cnn.get_output(seq_len=seq_len, input_op=input_op, **cnn_params)
r = rnn.get_model(batch_size=batch_size, seq_len=seq_len, input_op=c, **rnn_params)
logits, pred = classifier.get_logits_and_pred(input_op=r.last_output, **fc_params)



CNN--cnn10x32
Tensor("CNN/conv_module/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)

RNN--rnn--steps100--sizes10
LSTMStateTuple(c=<tf.Tensor 'RNN/LSTM/dynamic_wrapper/rnn/while/Exit_2:0' shape=(?, 10) dtype=float32>, h=<tf.Tensor 'RNN/LSTM/dynamic_wrapper/rnn/while/Exit_3:0' shape=(?, 10) dtype=float32>)

FC--fc
Tensor("classifier/fully_connected/logits/BiasAdd:0", shape=(?, 4), dtype=float32)
Tensor("classifier/pred:0", shape=(?, 4), dtype=float32)


# Time measure

convenience function

In [3]:
def measure_time(op, feed_dict={}, n_times=10):
    with tf.Session() as sess:
        print('Sess started')
        coord = tf.train.Coordinator()
        tf.global_variables_initializer().run()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        init_state = sess.run(r.zero_state)
        rnn_feed = {r.init_state : init_state}
        feed_dict.update(rnn_feed)
        print('Evaluating')
        for _ in range(n_times):
            t = time.time()
            test_output = sess.run(op, feed_dict)
            print(test_output, 'Eval time:', time.time() - t)
            
        print('Closing threads')
        coord.request_stop()
        coord.join(threads)

        return test_output

# Evaluation

## **Confusion matrix**

## **Accuracy operator**

In [4]:
x = tf.placeholder_with_default(input=logits, shape=[None, 4])
x_oh = tf.cast(tf.equal(x, tf.reduce_max(x, axis=1)[:, None]), tf.float32)[..., None]
#x_oh = tf.equal(x, tf.reduce_max(x, axis=1)[:, None])[..., None]
y = tf.placeholder_with_default(input=label, shape=[None])
y_oh = tf.one_hot(y, depth=4)[..., None]

In [5]:
conf_op = tf.reduce_sum(tf.transpose(x_oh, perm=[0, 2, 1]) * y_oh,
    axis=0, name='confusion_matrix')

print(conf_op)
x_tot = tf.reduce_sum(conf_op, axis=0)
y_tot = tf.reduce_sum(conf_op, axis=1)
correct_op = tf.diag_part(conf_op)
eps = [1e-10] * 4
acc_op = tf.reduce_mean(2*correct_op / (x_tot + y_tot + eps), name='accuracy')
print(acc_op)

Tensor("confusion_matrix:0", shape=(4, 4), dtype=float32)
Tensor("accuracy:0", shape=(), dtype=float32)


In [6]:
with tf.Session() as sess:
    print(conf_op.eval({x:[[.1, .2, .3, .4], [.1, .2, .3, .4]], y:[3, 0]}))
    print(sess.run([acc_op], {x:[[.1, .2, .3, .4], [.1, .2, .3, .4]], y:[3, 0]}))

[[ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]]
[0.16666667]


In [7]:
measure_time(acc_op, n_times=3)

Sess started
Evaluating
0.109756 Eval time: 2.7907462120056152
0.127002 Eval time: 0.49909067153930664
0.0681818 Eval time: 0.5650448799133301
Closing threads


0.06818182

# Sparse, weighted softmax loss

In [14]:
class_hist = np.load('./data/class_hist.npy')
weight = tf.gather(tf.constant(1 - np.sqrt(class_hist/class_hist.sum())), label, name='weight')

class_hist, weight

(array([5154,  771, 2557,   46]),
 <tf.Tensor 'weight:0' shape=(?,) dtype=float64>)

In [15]:
loss = tf.losses.sparse_softmax_cross_entropy(label, logits, weight)
unweighted_loss = tf.losses.sparse_softmax_cross_entropy(label, logits)

In [16]:
measure_time([loss, unweighted_loss])

Sess started
Evaluating
[0.49806154, 1.3993574] Eval time: 0.6323027610778809
[0.41979244, 1.3813533] Eval time: 0.4587879180908203
[0.49946821, 1.3793981] Eval time: 0.453294038772583
[0.52182496, 1.3881139] Eval time: 2.2066774368286133
[0.49599224, 1.3609421] Eval time: 0.4102365970611572
[0.40283754, 1.3684117] Eval time: 0.5056233406066895
[0.51788628, 1.395148] Eval time: 0.4189016819000244
[0.44691628, 1.3935302] Eval time: 0.4730651378631592
[0.45892927, 1.4054298] Eval time: 0.45441389083862305
[0.42116034, 1.3878207] Eval time: 0.4829261302947998
Closing threads


[0.42116034, 1.3878207]

# Train operator

In [17]:
learning_rate = tf.Variable(initial_value=.05, trainable=False, name='learning_rate')
global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
grad_clip = tf.Variable(initial_value=3., trainable=False, name='grad_clip')

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    gvs = optimizer.compute_gradients(loss)
    with tf.name_scope('gradient_clipping'):
        capped_gvs = [(tf.clip_by_value(grad, -grad_clip, grad_clip), var) 
                      for grad, var in gvs]
        
    opt = optimizer.apply_gradients(capped_gvs, global_step)

In [18]:
measure_time([loss, unweighted_loss, acc_op, opt], n_times=10)

Sess started
Evaluating
[0.48074871, 1.4008858, 0.055555556, None] Eval time: 3.2106688022613525
[0.38809419, 1.2457941, 0.21938775, None] Eval time: 1.535161018371582
[0.42092943, 1.1100838, 0.17897727, None] Eval time: 1.5493321418762207
[0.43511623, 1.1471479, 0.12790698, None] Eval time: 2.9214348793029785
[0.42858675, 1.1850848, 0.1097561, None] Eval time: 1.5219848155975342
[0.31411856, 1.0774897, 0.1, None] Eval time: 1.5246484279632568
[0.51634991, 1.1116172, 0.16666667, None] Eval time: 1.5631794929504395
[0.33072644, 0.93621349, 0.2090909, None] Eval time: 1.5349981784820557
[0.33033174, 0.91691887, 0.1981132, None] Eval time: 1.5619332790374756
[0.29583973, 0.86346889, 0.2037037, None] Eval time: 1.4362695217132568
Closing threads


[0.29583973, 0.86346889, 0.2037037, None]